In [5]:
import folium
import pandas as pd
import os

In [12]:
path=os.getcwd()
path=path[:-9]
path=path+'data/raw/corona/se_corona.csv'
corona=pd.read_csv(path,low_memory=False)

In [17]:
corona

,date\tregion_code\tconfirmed_addition
0,2021-02-18\tBlekinge\t23
1,2021-02-17\tBlekinge\t30
2,2021-02-16\tBlekinge\t50
3,2021-02-15\tBlekinge\t3
4,2021-02-14\tBlekinge\t3
...,...
7996,2020-02-08\tÖstergötland\t0
7997,2020-02-07\tÖstergötland\t0
7998,2020-02-06\tÖstergötland\t0
7999,2020-02-05\tÖstergötland\t0


In [15]:
path=os.getcwd() #'something\\somthingelse\\fyp2021\\notebooks'
path=path[:-9] #'something\\somthingelse\\fyp2021\\'
path=path+'data/raw/metadata/se_metadata.json' #'something/somthingelse/fyp2021/data\\raw\\corona\\se_corona.csv'

In [14]:
path=os.getcwd()
path=path[:-9]
path=path+'data/raw/shapesfiles/se.geojson'

In [18]:
path=os.getcwd()
path=path[:-9]
path=path+'data/raw/weather/weather.csv'
weather=pd.read_csv(path,low_memory=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=24195024-9b97-4fbb-b643-b1da4ede0b31' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>